# Serap YALÇIN
NLP Course Project 
29th March 2020 

In [14]:
import numpy as np
import pandas as pd
import re
import json
from pprint import pprint
from snowballstemmer import TurkishStemmer
import nltk
nltk.download("stopwords")
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 1- Read the dataset and parse it (i.e. tokenization, stemming, stopwords removal). (10+10= 20% of the assignment)


In [0]:
##READ the Datasets 
##data=pd.read_excel(r"C:\Users\budaks\Desktop\YL 2020 Bahar\NLP\MidtermProject\cv_listesi.xlsx")
from google.colab import files
uploaded = files.upload()



In [0]:
data=pd.read_excel('top15.xlsx')

In [6]:
##READ the Datasets 

data.columns = ['TermId', 'Name','description','Level1Domain']
data.head()

,TermId,Name,description,Level1Domain
0,1,kod,bankacılık kavramlarının sistem genelinde teki...,4585953
1,3,ad,bankacılık kavramlarını isimlendirmek için kul...,4585953
2,4,işlem tarihi,bankacılık işlemlerinin gercekleştirilme tarih...,4585953
3,5,bugünün tarihi,tüm bankacılık işlemleri için bugünün tarihini...,4585953
4,6,dünün tarihi,tüm bankacılık işlemleri için dünün tarihini g...,4585953


In [7]:
len(data)

31

In [0]:
from google.colab import files
uploaded = files.upload()


In [0]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from snowballstemmer import stemmer
import nltk


In [0]:
##Create functions to remove punctuation, tokenize, remove stopwords, and stem
turkish_stops=set(stopwords.words("turkish"))
#stopwords = set(stopwords.words('english'))
def clean_text(statement):
    
    tokenizer = RegexpTokenizer(r"\w+")
    statement_tokens = tokenizer.tokenize(str(statement))

   ## ps = nltk.PorterStemmer()
    ##TurkStem=TurkishStemmer()
    TurkStem=stemmer('turkish') #snowballstemmer
    
    statements_clean = []     

    for word in statement_tokens:
        if word not in turkish_stops:
            #if word not in stopwords:
             stem_word = TurkStem.stemWord(word)
             statements_clean.append(stem_word)
    return statements_clean

In [0]:
data["descriptionClean"]=data["description"].map(clean_text)
data["NameClean"]=data["Name"].map(clean_text)


In [0]:
data['descriptionClean2'] = data['descriptionClean']
data['NameClean2'] = data['NameClean']

In [0]:
import torch

In [0]:
!pip install -U sentence-transformers

In [0]:
from sentence_transformers import SentenceTransformer

In [24]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:05<00:00, 76.8MB/s]


In [0]:
from sklearn.cluster import KMeans

embedder = SentenceTransformer('bert-base-nli-mean-tokens')


In [0]:

# Corpus with example sentences
corpus = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'A man is eating pasta.',
          'The girl is carrying a baby.',
          'The baby is carried by the woman',
          'A man is riding a horse.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.',
          'A cheetah is running behind its prey.',
          'A cheetah chases prey on across a field.'
          ]
corpus_embeddings = embedder.encode(corpus)

# Perform kmean clustering
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [29]:
type(corpus)

list

In [36]:
type((data["description"]))

pandas.core.series.Series

In [0]:
data_embeddings = embedder.encode((data["description"]))

In [0]:
data_embeddings

In [0]:
num_clusters = 25
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(data_embeddings)
cluster_assignment = clustering_model.labels_



In [40]:
cluster_assignment

array([9, 6, 1, 1, 1, 6, 4, 5, 7, 7, 0, 0, 4, 0, 3, 0, 6, 6, 1, 8, 2, 4,
       6, 0, 6, 7, 6, 0, 4, 0, 0], dtype=int32)

In [0]:
data["label"] =  cluster_assignment

In [48]:
data

,TermId,Name,description,Level1Domain,descriptionClean,NameClean,descriptionClean2,NameClean2,label
0,1,kod,bankacılık kavramlarının sistem genelinde teki...,4585953,"[bankacılık, kavram, siste, genel, tekil, ayır...",[kod],"[bankacılık, kavram, siste, genel, tekil, ayır...",[kod],5
1,3,ad,bankacılık kavramlarını isimlendirmek için kul...,4585953,"[bankacılık, kavram, isimlendirmek, kullanıla,...",[ad],"[bankacılık, kavram, isimlendirmek, kullanıla,...",[ad],19
2,4,işlem tarihi,bankacılık işlemlerinin gercekleştirilme tarih...,4585953,"[bankacılık, işlem, gercekleştirilme, tarih, g...","[işle, tarih]","[bankacılık, işlem, gercekleştirilme, tarih, g...","[işle, tarih]",4
3,5,bugünün tarihi,tüm bankacılık işlemleri için bugünün tarihini...,4585953,"[bankacılık, işlem, bugü, tarih, göstere, ver]","[bugü, tarih]","[bankacılık, işlem, bugü, tarih, göstere, ver]","[bugü, tarih]",16
4,6,dünün tarihi,tüm bankacılık işlemleri için dünün tarihini g...,4585953,"[bankacılık, işlem, dü, tarih, göstere, ver]","[dü, tarih]","[bankacılık, işlem, dü, tarih, göstere, ver]","[dü, tarih]",16
5,7,yarının tarihi,tüm bankacılık işlemleri için yarının tarihini...,4585953,"[bankacılık, işlem, yar, tarih, göstere, ver]","[yar, tarih]","[bankacılık, işlem, yar, tarih, göstere, ver]","[yar, tarih]",16
6,8,ıd,her bir kayıt için sistem tarafından oluşturul...,4585953,"[bir, kayıt, siste, taraf, oluşturula, benzers...",[ıd],"[bir, kayıt, siste, taraf, oluşturula, benzers...",[ıd],17
7,9,kanal,müşteri ile kuveyt türk arasında köprü olan he...,4585953,"[müşter, kuveyt, türk, ara, köprü, ola, türlü,...",[kanal],"[müşter, kuveyt, türk, ara, köprü, ola, türlü,...",[kanal],6
8,11,ithalat,bir ülkede yerleşik kişi ve kurumların diğer ü...,4586010,"[bir, ülke, yerleşik, kiş, kur, diğer, ülke, m...",[ithalat],"[bir, ülke, yerleşik, kiş, kur, diğer, ülke, m...",[ithalat],7
9,12,tarih,bir işlemin gününü ayını ve yılını bildiren ve...,4585953,"[bir, işle, gün, ay, yıl, bildire, ver]",[tarih],"[bir, işle, gün, ay, yıl, bildire, ver]",[tarih],13
